In [5]:
pip install xgboost

ERROR: Exception:
Traceback (most recent call last):
  File "c:\Users\Elshaday\AppData\Local\Programs\Python\Python39\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "c:\Users\Elshaday\AppData\Local\Programs\Python\Python39\lib\site-packages\pip\_vendor\urllib3\response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "c:\Users\Elshaday\AppData\Local\Programs\Python\Python39\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "c:\Users\Elshaday\AppData\Local\Programs\Python\Python39\lib\http\client.py", line 462, in read
    n = self.readinto(b)
  File "c:\Users\Elshaday\AppData\Local\Programs\Python\Python39\lib\http\client.py", line 506, in readinto
    n = self.fp.readinto(b)
  File "c:\Users\Elshaday\AppData\Local\Programs\Python\Python39\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "c:\Users\Elshaday

In [1]:
import xgboost

ModuleNotFoundError: No module named 'xgboost'

In [1]:
import cv2
import mediapipe as mp
from math import hypot
import numpy as np
import pygetwindow as gw
import pyautogui
import time

In [3]:
# --- MediaPipe setup ---
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.75,
    min_tracking_confidence=0.75,
    max_num_hands=1
)
Draw = mp.solutions.drawing_utils

# --- Gesture thresholds (in pixels) -- 
up_val = 10    # below this = “pinch” → scroll down
down_val  = 10    # above this = “unpinch” → scroll up
DEBOUNCE_MS = 1000   # ignore new scrolls for this many ms

last_scroll_time = 0

# --- Camera setup ---
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Could not open webcam")

def find_pdf_window():
    """Return the first window ending with '.pdf', or None."""
    for w in gw.getAllTitles():
        if w and '.pdf' in w.lower():
            print("window found")
            return gw.getWindowsWithTitle(w)[0]
    return None

# wait until user opens a PDF
print("Waiting for a PDF window to appear…")
pdf_win = None
while pdf_win is None:
    pdf_win = find_pdf_window()
    time.sleep(0.5)
print(f"Found PDF: '{pdf_win.title}'")
pdf_win.activate()  # bring it to front
print("--- Main loop ---")
# --- Main loop ---
p1_0 = 0
p2_0 = 0
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.flip(frame, 1)
        frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        result = hands.process(frameRGB)
        pts = []
        if result.multi_hand_landmarks:
            handlm = result.multi_hand_landmarks[0]
            h, w, _ = frame.shape
            for idx, lm in enumerate(handlm.landmark):
                x_px, y_px = int(lm.x * w), int(lm.y * h)
                pts.append((idx, x_px, y_px))
            Draw.draw_landmarks(frame, handlm, mpHands.HAND_CONNECTIONS)

        # if we have thumb (4) and index (8)
        if pts:
            # build a dict for easy lookup
            d = {idx:(x,y) for idx,x,y in pts}
            if 12 in d and 8 in d:

                x1,y1 = d[8]
                x2,y2 = d[12]
                
                cv2.circle(frame, (x1,y1), 8, (0,255,0), cv2.FILLED)
                cv2.circle(frame, (x2,y2), 8, (0,255,0), cv2.FILLED)

                L = hypot(x2 - x1, y2 - y1)
                print("got L")
                p1 = hypot(x1, y1)
                p2 = hypot(x2, y1)
                now = time.time()*1000
                #time.sleep(0.1)
                if now - last_scroll_time > DEBOUNCE_MS:

                    print("got p")
                    print(p1 - p1_0)
                    if np.abs(p2 - p2_0) > down_val:
                        pyautogui.scroll(-300)   # scroll down
                        last_scroll_time = now
                        print("scroll_down")
                        p2_0 = p2
                    elif np.abs(p1 - p1_0) > up_val:
                        pyautogui.scroll(300)    # scroll up
                        last_scroll_time = now
                        print("scroll_up")
                        p1_0 = p1

        cv2.imshow('PDF Scroll Control', frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break

finally:
    cap.release()
    cv2.destroyAllWindows()


Waiting for a PDF window to appear…
window found
Found PDF: 'materials-16-02692.pdf - WPS Office'
--- Main loop ---
got L
got p
477.6023450528693
scroll_down
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got p
390.60977970347847
scroll_down
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got p
386.2447410645225
scroll_up
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got p
-0.6322412671222537
got L
got p
-1.5370598310956325
got L
got p
-2.013040133951222
got L
got p
-6.180272848931452
got L
got p
-5.159448774200655
got L
got p
0.11390061442324395
scroll_down
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got p
18.786121957050284
scroll_down
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got L
got p
19.760185013967373
scroll_up
got L
